In [59]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 3


URL = 'https://www.misssixty.com/us_en/ready-to-wear/ready-to-wear-autum-winter/view-all.html'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
        
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source,'html.parser')

picture_url = []
product_url = []
brand = []
product_type = []
timestamp = []
name = []
price = []
sales_price = []

for i in soup.find_all('div',{'class':'product configurable col-md-3 col-6'}):
    
    brand.append('Miss Sixty')
    timestamp.append(date_today)
    product_type.append("Women's Clothing")
    
    product_url.append(i.a.get('href'))
    picture_url.append(i.a.img.get('data-src'))
    
    name.append(i.find('a',{'class':'product-name'}).get('title'))

    if len(i.find('div',{'class':'price-box'}).find('div',{'class':'price'}).get('class'))==1:
        sales_price.append(i.find('div',{'class':'price-box'}).find('div',{'class':'price'}).text)
        price.append(None)
    else:
        sales_price.append(i.find('div',{'class':'price-box'}).find('div',{'class':'final-price'}).text)
        price.append(i.find('div',{'class':'price-box'}).find('div',{'class':'price old-price'}).text)

# Second Layer

In [56]:
product_description = []
material = []
picture_name = []
color = []
counter = 1

for link in product_url:
    item_color = []
    item_material = []
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    for color_i in soup.find('div',{'class':'product-color-wrapper'}).find_all('img'):
        item_color.append(color_i.get('alt'))
    color.append(item_color)

    for des_i in soup.find('div',{'class':'product-info'}).find_all('div',{'class':'accordion'}):
        item_material.append(des_i.text.strip())
    material.append([item_material[1]])
    product_description.append(item_material[0])
    
    picture_name.append('miss-sixty_' + str(counter) + '.jpg') # change variable
    counter += 1

In [61]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Miss Sixty,Women's Clothing,White jacket with real fur,FJ0770 JACKET WHITE L,"$1,330.00",$931.00,[White],[COTTON 100%],https://www.misssixty.com/us_en/fj0770-jacket-...,https://www.misssixty.com/media/catalog/produc...,2020-12-07,miss-sixty_1.jpg
1,Miss Sixty,Women's Clothing,High-waisted jeans with slim fit,JJ2200 JEANS,None,$189.00,[Blue],"[COTONE 94,30 %, POLIESTERE 4,20 %, ELASTAN 1,...",https://www.misssixty.com/us_en/jj2200-jeans-4...,https://www.misssixty.com/media/catalog/produc...,2020-12-07,miss-sixty_2.jpg
2,Miss Sixty,Women's Clothing,Low-waist washed jeans with raw hem,JJ4500 JEANS,$119.00,$83.30,[Black],"[COTONE 91,40 %, POLIESTERE 6,40 %, ELASTAN 2,...",https://www.misssixty.com/us_en/jj4500-jeans-4...,https://www.misssixty.com/media/catalog/produc...,2020-12-07,miss-sixty_3.jpg
3,Miss Sixty,Women's Clothing,Faux fur bomber jacket,WJ3860 JACKET,$279.00,$195.30,[Beige],"[POLYESTER SYNTHETIC FUR 100,00 %]",https://www.misssixty.com/us_en/wj3860-jacket....,https://www.misssixty.com/media/catalog/produc...,2020-12-07,miss-sixty_4.jpg
4,Miss Sixty,Women's Clothing,Faux fur biker style jacket,WJ4360 JACKET,$379.00,$265.30,[Red],"[POLYESTER SYNTHETIC FUR 100,00 %]",https://www.misssixty.com/us_en/wj4360-jacket....,https://www.misssixty.com/media/catalog/produc...,2020-12-07,miss-sixty_5.jpg


In [62]:
second_pass.to_csv("miss-sixty.csv", index=False, encoding = 'utf-8-sig')

# Download Images

In [63]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/miss-sixty.csv', encoding='utf-8-sig')

os.mkdir('miss-sixty')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('miss-sixty/'+ "miss-sixty_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1